<a href="https://colab.research.google.com/github/NongNNew/Project_499/blob/main/Project_499_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN

In [ ]:
!git clone https://github.com/NongNNew/Project_499.git

Cloning into 'Project_499'...
remote: Enumerating objects: 823, done.
remote: Counting objects: 100% (375/375), done.
remote: Compressing objects: 100% (366/366), done.
remote: Total 823 (delta 34), reused 334 (delta 9), pack-reused 448
Receiving objects: 100% (823/823), 139.08 MiB | 6.71 MiB/s, done.
Resolving deltas: 100% (62/62), done.
Checking out files: 100% (662/662), done.


In [ ]:
fruit_veget = ['กระชาย','กระท้อน','กระเทียม1','กระเทียม2','กระเพรา','กล้วยน้ำว้า','กล้วยหอม',
               'ข้าวโพด','ไข่น้ำ',
               'ตะขบไทย','ตะไคร้',
               'ถั่วฝักยาว','ถั่วลันเตา','ถั่วลิสง',
               'ทับทิม',
               'น้อยหน่า','น้ำเต้า',
               'ผักกระเฉด','ผักกุยช่าย',
               'บวบ','ใบชะพลู1','ใบชะพลู2','ใบบัวบก','ใบแมงลัก','ใบยอ',
               'พริกขี้หนู','พริกสด',
               'ฝรั่ง',
               'พุทรา',
               'มะปราง','มะพลับ','มะละกอ',
               'สับปะรด']

source = {'audio_time':[],
          'sampling_rate':[],
          'feature_extraction':[],
          'label':[],
          'description':[]}

In [ ]:
import pandas as pd
from glob import glob
import librosa

for types in fruit_veget:
    audio_files = glob('/content/Project_499/Record_audio/' + str(types) + '/*.wav')

    for audio in audio_files:
        y,sr = librosa.load(audio,duration=6,offset=0)
        source['description'].append(str(types))
        source['audio_time'].append(y)
        source['sampling_rate'].append(sr)
        source['feature_extraction'].append(librosa.feature.mfcc(y,sr,n_mfcc=13))
        source['label'].append(fruit_veget.index(types))

In [ ]:
df = pd.DataFrame.from_dict(source)
df

,audio_time,sampling_rate,feature_extraction,label,description
0,"[-0.0001373291, -0.00010681152, -0.0001373291,...",22050,"[[-641.21436, -640.2155, -639.54114, -638.4883...",0,กระชาย
1,"[0.00015258789, 0.00022888184, 0.00022888184, ...",22050,"[[-638.9144, -641.23175, -641.37787, -636.0402...",0,กระชาย
2,"[-7.6293945e-05, -0.00015258789, -0.0001068115...",22050,"[[-670.6008, -670.93756, -673.4263, -673.5513,...",0,กระชาย
3,"[1.5258789e-05, -3.0517578e-05, 6.1035156e-05,...",22050,"[[-635.4628, -636.8742, -637.42847, -636.0504,...",0,กระชาย
4,"[-0.0002593994, -0.00021362305, -0.0002593994,...",22050,"[[-677.20435, -674.49866, -673.315, -679.4692,...",0,กระชาย
...,...,...,...,...,...
655,"[-4.5776367e-05, -6.1035156e-05, -0.0001525878...",22050,"[[-616.3849, -615.21027, -615.2758, -615.1418,...",32,สับปะรด
656,"[-3.0517578e-05, -1.5258789e-05, -0.0001983642...",22050,"[[-664.07837, -662.51434, -660.61566, -660.430...",32,สับปะรด
657,"[0.0, 0.0, 0.0, 3.0517578e-05, -4.5776367e-05,...",22050,"[[-644.81506, -644.89075, -644.7345, -643.6741...",32,สับปะรด
658,"[-7.6293945e-05, -3.0517578e-05, -1.5258789e-0...",22050,"[[-620.5951, -621.67816, -622.1715, -620.5547,...",32,สับปะรด


In [ ]:
import numpy as np
import pandas as pd

Test = pd.DataFrame(columns=source.keys())

# create Test model
for description in np.unique(df['description']):
    imp = df[df['description']==description].sample(5,random_state=1)
    Test = pd.concat([Test,imp], axis=0)

# create Train model
Train = df.drop(Test.index)

X_train, X_test = np.array(Train['feature_extraction'].to_list()), np.array(Test['feature_extraction'].to_list())
y_train, y_test = np.array(Train['label'].to_list()), np.array(Test['label'].to_list())

In [109]:
import tensorflow as tf
from tensorflow import keras 
np.random.seed(42)
tf.random.set_seed(42)

model = keras.Sequential()
model.add(keras.layers.Conv2D(128, 5, input_shape=(13, 259, 1), activation='relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Conv2D(128, 5, activation='relu',padding='same'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(len(np.unique(source['label'])), activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train[:,:,:,None], y_train, epochs=100)

Epoch 1/100
16/16 [==============================] - 1s 19ms/step - loss: 34.1658 - accuracy: 0.0283
Epoch 2/100
16/16 [==============================] - 0s 14ms/step - loss: 3.3212 - accuracy: 0.1030
Epoch 3/100
16/16 [==============================] - 0s 14ms/step - loss: 3.0526 - accuracy: 0.1354
Epoch 4/100
16/16 [==============================] - 0s 14ms/step - loss: 2.5364 - accuracy: 0.3071
Epoch 5/100
16/16 [==============================] - 0s 14ms/step - loss: 2.1539 - accuracy: 0.3899
Epoch 6/100
16/16 [==============================] - 0s 14ms/step - loss: 1.9333 - accuracy: 0.4101
Epoch 7/100
16/16 [==============================] - 0s 14ms/step - loss: 1.7259 - accuracy: 0.4808
Epoch 8/100
16/16 [==============================] - 0s 14ms/step - loss: 1.2946 - accuracy: 0.5899
Epoch 9/100
16/16 [==============================] - 0s 14ms/step - loss: 1.1954 - accuracy: 0.6485
Epoch 10/100
16/16 [==============================] - 0s 14ms/step - loss: 1.0065 - accuracy: 0.686

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 255, 64)        1664      
_________________________________________________________________
dropout (Dropout)            (None, 9, 255, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 127, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 127, 64)        102464    
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 127, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 63, 64)         0         
_________________________________________________________________
flatten (Flatten)            (None, 8064)              0

In [110]:
model.evaluate(X_test[:,:,:,None],y_test)

6/6 [==============================] - 0s 5ms/step - loss: 0.3898 - accuracy: 0.9576


[0.3898232579231262, 0.9575757384300232]

In [111]:
from sklearn.metrics import classification_report

predicted_classes = np.argmax(np.round(model.predict(X_test[:,:,:,None])),axis=1)
target_names = [f"Class {label}" for label in range(len(np.unique(source['label'])))]
print(classification_report(y_test, predicted_classes, target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.83      1.00      0.91         5
     Class 1       1.00      1.00      1.00         5
     Class 2       1.00      1.00      1.00         5
     Class 3       1.00      0.60      0.75         5
     Class 4       1.00      1.00      1.00         5
     Class 5       1.00      1.00      1.00         5
     Class 6       1.00      1.00      1.00         5
     Class 7       0.83      1.00      0.91         5
     Class 8       1.00      1.00      1.00         5
     Class 9       1.00      0.60      0.75         5
    Class 10       0.83      1.00      0.91         5
    Class 11       1.00      1.00      1.00         5
    Class 12       1.00      0.80      0.89         5
    Class 13       1.00      1.00      1.00         5
    Class 14       1.00      1.00      1.00         5
    Class 15       0.83      1.00      0.91         5
    Class 16       1.00      1.00      1.00         5
    Class 17       0.83    